In [ ]:
%%time
# Importing all the necessary libraries for this project.

import pandas as pd
import numpy as np

In [ ]:
# VIEW ALL COLUMNS & ALL TEXT IN EACH CELL

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', None)

# Set ipython's max row display
pd.set_option('display.max_row', 1000)

# Set iPython's max column width to 50
pd.set_option('display.max_columns', 50)


### Import the files here. Here is a breakdown of what is inside each file:

- Whisky_pd. Preliminary information of bottle & the price
- whisky_details. Contains detailed information of the bottle/whisky
- final_review. Contains information of the reviews of each whisky bottle
- distillery_main. 

In [ ]:
## Import everything again.

whisky_pd = pd.read_csv("Whisky_Main.csv", index_col = 0)
final_review = pd.read_csv('Final_Review.csv', index_col=0)
whisky_details = pd.read_csv('Whisky Details.csv', index_col=0, dtype = 'object')
distillery_main = pd.read_csv('Distillery_Info.csv', index_col=0)

In [ ]:
## Merge details & whisky_pd on whisky_pd URL.

whisky_pd = pd.merge(whisky_pd, whisky_details, how = 'left', left_on = 'Link', right_on = 'WhiskyURL')

In [ ]:
## Merge Distillery with whisky_pd on DistilleryName

whisky_pd = pd.merge(whisky_pd, distillery_main, how = 'left', on = 'Distillery')

In [ ]:
whisky_pd.drop_duplicates(inplace = True)

## Let us start with some EDA on whisky_pd and fixing the data types

In [ ]:
## fix the volume (remove cl), remove pound sign/convert to $ and fix ABV by removing %

whisky_pd['Volume'] = whisky_pd['Volume'].apply(lambda x: float(x.replace('cl', ''))*10)
whisky_pd['Price/Bottle'] = whisky_pd['Price/Bottle'].apply(lambda x: float(x.replace('£', '').replace(',', ''))*1.14)
whisky_pd['ABV'] = whisky_pd['ABV'].apply(lambda x: float(x.replace('%', '')))

In [ ]:
## Rename the columns

whisky_pd = whisky_pd.rename(columns={'Volume' : 'Volume_ml', 'Price/Bottle' : 'Price_Bottle$', 'ABV': 'ABV'})

In [ ]:
## Normalize the prices here to 1L

whisky_pd['Price_L'] = round((whisky_pd['Price_Bottle$']*1000/whisky_pd['Volume_ml']),2)

In [ ]:
## Fix the Distillery names. Removing preceeding & trailing white spaces

whisky_pd['Distillery'] = whisky_pd['Distillery'].str.rstrip().str.lstrip()
whisky_pd['Whisky'] = whisky_pd['Whisky'].str.rstrip().str.lstrip()

In [ ]:
## Removing miniatures, kegs & only >40ABV whisky. we will focus on whisky that are >500ml and under 1L.

whisky_pd = whisky_pd[(whisky_pd['Volume_ml'] > 500) & (whisky_pd['ABV'] >= 40.0) & (whisky_pd['Volume_ml']<1000)].reset_index(drop = True)

In [ ]:
## Creating Categories of Whisky. Remember, these prices are per L & typically bottles are ~700ml

def categorizer(price):
    if price < 71.5:
        return 0
    elif (price >=71.5) & (price < 142.85):
        return 1
    elif (price >=142.85) & (price < 1428.5):
        return 2
    else:
        return 3

In [ ]:
def bincategorizer(price):
    if price <= 715:
        return 0
    else:
        return 1

In [ ]:
whisky_pd.dropna(axis=0, how='all', subset=['Price_L'], inplace=True)

### CREATING WHISKY CLASSES. THE PRICES ARE BASED OFF DISCUS (Distilled Spirit Council of USA). Their definition is not perfect and prices are based off supplier/bulk rates. So we are using the same approximation here.

#### Value : Price < $50/bottle$ - 0
#### Premium : Price < $100/bottle$ - 1
#### High End : Price < $1000/bottle$ - 2
#### Super Premium : Price > $1000/bottle$ - 3

### CREATING AN ALTERNATE CLASS (AFFORDABLE VS EXPENSIVE):

#### Affordable : Price < $200/bottle$ - 0
#### HighEnd : Price > $200/bottle$ - 1

In [ ]:
## Creating class for whisky. 

whisky_pd['Class'] = whisky_pd['Price_L'].apply(categorizer)

In [ ]:
whisky_pd['BinClass'] = whisky_pd['Price_L'].apply(bincategorizer)

In [ ]:
whisky_pd.BinClass.value_counts()

In [ ]:
## Drop the 0 column. It is full of null values. Don't need it.

whisky_pd.drop('0', inplace = True, axis = 1)

In [ ]:
## Creating Age statements on the sample.

new_column = []

for index, values in whisky_pd[['Whisky', 'WhiskyURL', 'Age']].iterrows():
    try: 
        new_column.append(float(values[2].split()[0]))
    except:
        try:
            yearsum = min([float(s) for s in values[0].split() if s.isdigit()])
            if (yearsum < 100 & yearsum>3):
                new_column.append(yearsum)
            else:
                new_column.append(np.NaN)
        except:
            try:
                yearsum = min([float(s) for s in values[0].split('-') if s.isdigit()])
                if (yearsum < 100 & yearsum>3):
                    new_column.append(yearsum)
                else:
                    new_column.append(np.NaN)
            except:
                new_column.append(np.NaN)

whisky_pd['WhiskyAge'] = new_column

In [ ]:
whisky_pd['AgeType'] = np.where(whisky_pd['WhiskyAge'].isnull(), 'NAS', 'YAS')

In [ ]:
### Fix the Age/WhiskyAge column and drop one here

whisky_pd['WhiskyAge'] = whisky_pd['WhiskyAge'].astype(float, inplace = True)

In [ ]:
## DETAILS IS DICTIONARY THAT HAS BEEN OPENED INTO COLUMNS, LINK = PAGE LINK, BOTTLENAME SAME AS WHISKYNAME

whisky_pd.drop(['Details', 'Link', 'BottleName'], axis = 1, inplace = True)

In [ ]:
### DROP THE AGE COLUMN NOW THAT WE HAVE WHISKYAGE, AGETYPE, TITLEEXTRA

whisky_pd.drop("Age", axis = 1, inplace = True)

In [ ]:
## Removing some null & cleaning some more data! Removing null URLS & Distillerys

whisky_pd.dropna(axis=0, how='all', subset=['WhiskyURL', 'Distillery'], inplace=True)

In [ ]:
## Fix Country Column & drop the Whisky that have missing country

whisky_pd['Country'] = whisky_pd['Country'].fillna(method='ffill').fillna(method='bfill')

whisky_pd.dropna(axis=0, how='all', subset=['Country'], inplace=True)

In [ ]:
## Fix the colouring information

whisky_pd['Colouring'].fillna("No", inplace = True)

In [ ]:
## Let us start tackling the columns & missing values here by Deductive imputation

# Start by Character Column. If no character, replace NaN by 'Character'. We will fix this later in NLP in stop words

whisky_pd['TasteInfo'] = np.where(whisky_pd['Character'].isnull(), 'No Info', 'Available')

In [ ]:
## Due to a scraper problem, some of the descriptions died. Dropping the remaining descriptions.

whisky_pd[whisky_pd['Description'].isnull()]

whisky_pd.loc[275, 'Description'] = "Glenfiddich asked its 20 ambassadors to choose their favourite cask from the distillery's warehouses. From the character of these casks, malt master Brian Kinsman created a new, experimental whisky – Project XX. A complex vatting of casks, from soft and fruity ex-bourbon to rich port pipes and sherry butts, Project XX is a refined and well-balanced whisky with lots of depth."
whisky_pd.loc[488, 'Description'] = "In November 2013, we chose Glenfarclas 15 Year Old as our Whisky of the Year. In honour of this, we spoke to Glenfarclas about creating a Whisky Exchange exclusive edition. We decided on a different approach to the classic 15 Year Old: our exclusive edition is bottled at cask strength – 58.6% abv (103° proof in old money)."
whisky_pd.loc[2218, 'Description'] = "Introducing the pinnacle of the Chivas Regal range – The Icon. Veteran master blender Colin Scott has selected some of Chivas's rarest casks, including whiskies from long-closed distilleries, and created a blend that proudly demonstrates the skill of the whisky makers of Scotland. It is presented in a green crystal decanter, handmade and etched by the artisans at Dartington Crystal, dressed with detailed metalwork and sealed with a heavy stopper featuring Chivas famed luckenbooth marque."
whisky_pd.loc[2764, 'Description'] = "Consistently one of Indian distillery Amrut's most popular bottles, Fusion is a rich and warming single malt with delicious notes of fresh fruit and vanilla, followed by a smoky finish. Amrut Fusion is created from a mix of 75% unpeated Indian barley and 25% peated Scottish barley. These are separately distilled and aged for four years, then 'fused' together for a further three months. The result is an award-winning whisky that has introduced many people to the splendours of Indian whisky."
whisky_pd.loc[2471, 'Description'] = "Rich and full-bodied, Woodford Reserve is a delicious bourbon, triple distilled for extra smoothness, and loaded with notes of dried fruit and sweet spice. Ideal served as a digestif after dinner, on the rocks, or as the basis for a cocktail."

whisky_pd.dropna(axis = 0, how = 'all', subset = ['Description'], inplace = True)

In [ ]:
## DROP THE FLAVOR COLUMN SINCE IT HAS ONLY 2 ENTRIRES

whisky_pd.drop('Flavour', inplace = True, axis = 1)

In [ ]:
## Fixing the Regions & merging the regions from silent stills & TWE into one column. Then we drop the Region_y


whisky_pd['Region_x']=np.where(whisky_pd['Region_x'].isnull(), whisky_pd['Region_y'], whisky_pd['Region_x'])

whisky_pd.drop("Region_y", inplace = True, axis = 1)

whisky_pd = whisky_pd.rename(columns={'Region_x' : 'Region'})

In [ ]:
### Fix the whisky Status. We have added the list of all Silent Stills. The ones that are missing are active stills.

whisky_pd['Status'].fillna('Active', inplace = True)

In [ ]:
## REMOVE THE ENTRIES THAT ARE A "PACK" OR CONTAIN "GLASS SET"

whisky_pd = whisky_pd[whisky_pd.Whisky.str.contains("Glass") == False]
whisky_pd = whisky_pd[whisky_pd.Whisky.str.contains("Pack") == False]
whisky_pd = whisky_pd[whisky_pd.Whisky.str.contains("Miniature") == False]
whisky_pd = whisky_pd[whisky_pd.Whisky.str.contains("Set") == False]
whisky_pd = whisky_pd[whisky_pd.Whisky.str.contains("Tasting") == False]

In [ ]:
## Fixing the typos

whisky_pd['Bottler'].replace('Distilelry', 'Distillery', inplace = True, regex = True)
whisky_pd['Bottler'].replace('Distilery', 'Distillery', inplace = True, regex = True)

In [ ]:
## Fix the Bottler here. We are creating a list of independent bottlers and looping through them to fix the nan values
## if the bottler exists, then fix it. else ignore.

whisky_pd['Bottler'] = whisky_pd['Bottler'].str.rstrip()
bottlers = list(whisky_pd['Bottler'].dropna().unique())


bottler = []

for index, values in whisky_pd[['Whisky', 'Bottler', 'Distillery']].iterrows():
    if type(values[1])==float:
        for i in bottlers:
            if i in values[0]:
                bottler.append(i)
                break
            else:
                bottler.append('Distillery Bottling')
                break
    else:
        bottler.append(values[1])
        
whisky_pd['NewBottler'] = bottler

whisky_pd['Bottler']=np.where(whisky_pd['Bottler'].isnull(), whisky_pd['NewBottler'], whisky_pd['Bottler'])
whisky_pd.drop('NewBottler', inplace = True, axis = 1)


## Now merge and remove NaN values

In [ ]:
whisky_pd['BottlingType'] = np.where(whisky_pd['Bottler']=='Distillery Bottling', 'Distillery', 'Independent')

In [ ]:
## Pull out vintage from Description or WhiskyName

vintagebot = []

for index, values in whisky_pd[['Vintage', 'Whisky', 'Description']].iterrows():
    
    ## check if vintage is nan. if it is then -- go to description and pull largest number > 1900
    if type(values[0]) == float:
        sequence = [int(s) for s in (values[1] + values[2]).split() if s.isdigit()]
        if len(sequence)>0:
            if max(sequence)>1900:
                vintagebot.append(max(sequence))
            else:
                vintagebot.append(np.NaN)
        else:
            vintagebot.append(np.NaN)
    else:
        vintagebot.append(int(values[0]))

## Bringing it all together here.
        
        
whisky_pd['WhiskyVintage'] = vintagebot


## Create a WhiskyVintage column showing if Vintage is available or not

whisky_pd['VintageInfo'] = np.where(whisky_pd['WhiskyVintage'].isnull(), 'No Vintage', 'Vintage')
whisky_pd.drop('Vintage', axis = 1, inplace = True)

In [ ]:
whisky_pd['LimitedEditions'] = np.where(whisky_pd['No of Bottles'].isnull(), 'No Info', 'Limited')

In [ ]:
whisky_pd['CaskInfo'] = np.where(whisky_pd['Cask Number'].isnull(), 'No Info', 'SpecialCask')

In [ ]:
## Drop Type column containing only 76 entries. Not a good enough predictor

whisky_pd.drop('Type', inplace = True, axis = 1)

In [ ]:
whisky_pd['#Reviews'].fillna(0, inplace = True)

In [ ]:
whisky_pd['Region'] = whisky_pd['Region'].combine_first(whisky_pd['Country'])

In [ ]:
whisky_pd.to_csv('Whisky_EDA.csv', header = True)

## Let us start with some EDA on whisky_details and fixing the data types

In [ ]:
final_review.reset_index(drop=True, inplace = True)

In [ ]:
final_review[final_review['Link']=='https://www.thewhiskyexchange.com/p/2562/glen-moray-classic']